In [1]:
import time
t1 = time.time()

import math
import os
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten , Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate, Input
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
from sklearn.metrics import confusion_matrix
import tensorflow.keras.backend as K
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

2025-05-19 05:26:19.238208: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
##### load image data #####

image_path = "/root/data/data_image.h5"
image_file = h5py.File(image_path, 'r')
image_file.keys()

<KeysViewHDF5 ['jjBG', 'sig', 'ttbarBG']>

In [3]:
sig_image = image_file['sig'][:]
print("signal shape", np.shape(sig_image))

jjBG_image = image_file['jjBG'][:]
print("jjBG shape", np.shape(jjBG_image))

ttbarBG_image = image_file['ttbarBG'][:]
print("ttbarBG shape", np.shape(ttbarBG_image))

total_image = np.concatenate((sig_image, jjBG_image, ttbarBG_image))
print("total image shape", np.shape(total_image))

signal shape (174453, 50, 50)
jjBG shape (123337, 50, 50)
ttbarBG shape (123751, 50, 50)
total image shape (421541, 50, 50)


In [4]:
##### make target #####

target = np.zeros(len(sig_image) + len(jjBG_image) + len(ttbarBG_image))
target[0:len(sig_image)] = 1    ### signal label as 1
target = to_categorical(target, num_classes=2)    ### signal label as [0, 1]
print(target)
print("target shape", np.shape(target))

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
target shape (421541, 2)


In [5]:
##### build CNN model #####



In [6]:
image_file.close()
t2 = time.time()
print("Total time (s): ", t2-t1)

Total time (s):  5.972450017929077
